### Import packages

In [1]:
!pip install ipympl
!pip install --upgrade numpy
!pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive 
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, recall_score, f1_score
import pandas as pd
import seaborn as sn
import math
from random import shuffle
from collections import Counter
import os
import h5py
# import ipdb # ipdb.set_trace() ### breakpoint
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib widget
#%matplotlib inline

Mounted at /content/drive


### Main parameters

In [2]:
n_classes = 10
n_clusters = 10
n_train = 3000 # max 5420 # full 7291
n_test = 1400 # max 1470 # full 2007

### Data loading

Load the data

In [3]:
current_dir = os.getcwd()
with h5py.File('drive/My Drive/Colab Notebooks/investigate_myelin_rule/usps/k_means/usps.h5', 'r') as hf:
        train = hf.get('train')
        train_X = train.get('data')[:]
        train_Y = train.get('target')[:]
        test = hf.get('test')
        test_X = test.get('data')[:]
        test_Y = test.get('target')[:]

# for j in range(10):
#     print(len(train_Y[train_Y == j]))
# print('\n')
# for j in range(10):
#     print(len(test_Y[test_Y == j]))

Data balancing

In [4]:
n_samples_train = n_train//n_classes
x_ = []
y_ = []
for ind in range(n_classes):
    x_.append(train_X[train_Y==ind][:n_samples_train])
    y_.append(train_Y[train_Y==ind][:n_samples_train])
y_ = np.array(y_).reshape(-1)
x_ = np.array(x_).reshape(-1,256)
data = list(zip(x_,y_))
shuffle(data)
suffled_data = list(zip(*data))
train_images = np.array(list(suffled_data[0]))
train_labels = np.array(list(suffled_data[1]))

n_samples_test = n_test//n_classes
x_ = []
y_ = []
for ind in range(n_classes):
    x_.append(test_X[test_Y==ind][:n_samples_test])
    y_.append(test_Y[test_Y==ind][:n_samples_test])
y_ = np.array(y_).reshape(-1)
x_ = np.array(x_).reshape(-1,256)
data = list(zip(x_,y_))
shuffle(data)
suffled_data = list(zip(*data))
test_images = np.array(list(suffled_data[0]))
test_labels = np.array(list(suffled_data[1]))

# for ind in range(10):
#     print(len(train_labels[train_labels == ind]))
# print('\n')
# for ind in range(10):
#     print(len(test_labels[test_labels == ind]))


Plot the data

In [ ]:
# plt.figure(figsize = (6, 2))
# plt.subplot(1,2,1)
# plt.imshow(train_images[0].reshape(16,16), cmap='gray')
# plt.title("image 0")
# plt.colorbar()
# plt.subplot(1,2,2)
# plt.imshow(train_images[7].reshape(16,16), cmap='gray')
# plt.title("image 1")
# plt.colorbar()

### Functions

In [5]:
def CreateClasses(model_labels, train_labels, n_classes, n_clusters):
    react_num = np.zeros([n_classes, n_clusters])
    classes_dic = {}
    n_samples = len(model_labels)
    for i in range(n_samples):
        react_num[train_labels[i], :] += np.eye(n_classes)[model_labels[i], :]
    for j in range(n_clusters): 
        classes_dic[j] = np.argmax(react_num[:, j])
    return classes_dic

def show_confusion_matrix(pred, labels):
    data = {'y_Actual': labels, 'y_Predicted': pred}
    df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
    confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
    plt.figure(figsize = (10, 8))
    sn.heatmap(confusion_matrix, annot=True)

# def show_wrong_pics(pred, labels, images):
#     is_wrong_ind = np.array(pred) != np.array(labels)
#     for j in range(len(is_wrong_ind)):
#         if is_wrong_ind[j]:
#             plt.figure(figsize = (4, 4))
#             plt.imshow(images[j].reshape(round(np.sqrt(ni)), round(np.sqrt(ni))), cmap='gray')
#             plt.xlabel(f"Actual: {labels[j]}, Predicted: {pred[j]}")
#             plt.title(f"Wrong recognized pic: {j}")
#             plt.colorbar()

def analyse(pred, labels):
    precision = accuracy_score(labels, pred)
    recall = recall_score(labels, pred, average='micro')
    F1_score = f1_score(labels, pred, average='micro')
    return precision, recall, F1_score 

### Training

In [6]:
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(train_images)
model_labels = kmeans.labels_

In [7]:
classes = CreateClasses(model_labels, train_labels, n_classes, n_clusters)
print(classes)

{0: 6, 1: 1, 2: 2, 3: 7, 4: 5, 5: 0, 6: 3, 7: 9, 8: 4, 9: 0}


In [8]:
cluster_centers = kmeans.cluster_centers_

# for j in range(n_clusters):
#     plt.figure(figsize = (4, 4))
#     plt.imshow(cluster_centers[j].reshape(16, 16), cmap='gray')
#     plt.title(f"Cluster center for pic {classes[j]}")
#     plt.colorbar()

### Testing

In [9]:
model_train_predictions = kmeans.predict(train_images)
predict_train_labels = [classes[model_pred] for model_pred in model_train_predictions]

model_test_predictions = kmeans.predict(test_images)
predict_test_labels = [classes[model_pred] for model_pred in model_test_predictions]

In [10]:
#show_confusion_matrix(predict_train_labels, train_labels)
precision, recall, F1_score = analyse(predict_train_labels, train_labels)
# show_wrong_pics(predict_train_labels, train_labels, train_images)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1_score: ", F1_score) 

Precision:  0.6383333333333333
Recall:  0.6383333333333333
F1_score:  0.6383333333333333


In [11]:
# show_confusion_matrix(predict_test_labels, test_labels)
precision, recall, F1_score = analyse(predict_test_labels, test_labels)
# show_wrong_pics(predict_test_labels, test_labels, test_images)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1_score: ", F1_score) 

Precision:  0.6321428571428571
Recall:  0.6321428571428571
F1_score:  0.6321428571428571
